In [ ]:
import pickle
from collections import defaultdict
import json
#import simplejson as json

In [ ]:
simp2trad=json.load(open('./simp2tra.json', 'r'))
tra2simp=json.load(open('./tra2simp.json', 'r'))


In [ ]:
#only keep keys with multiple entries with each entry's count >100
simp2multitrad={}
trad2multisimp={}
for key in simp2trad:
    if len(simp2trad[key].keys())>1:
        temp_dict={}
        for tra_key in simp2trad[key]:
            if int(simp2trad[key][tra_key])>1000:
                temp_dict[tra_key]=simp2trad[key][tra_key]
        if len(temp_dict)>1:
            simp2multitrad[key]=dict(temp_dict)
for key in tra2simp:
    if len(tra2simp[key].keys())>1:
        temp_dict={}
        for simp_key in tra2simp[key]:
            if int(tra2simp[key][simp_key])>1000:
                temp_dict[simp_key]=tra2simp[key][simp_key]
        if len(temp_dict)>1:
            trad2multisimp[key]=dict(temp_dict)
        
#num of simplified characters with multiple traditional
print (len(simp2multitrad))
#num of tra characters with multiple simplified
print (len(trad2multisimp))

#compile multitrad tra characters
multitrad={}
multisimp={}
for chars in simp2multitrad.values():
    multitrad.update(chars)
for chars in trad2multisimp.values():
    multisimp.update(chars)




In [ ]:
# process files to extract test cases, max 20 sentences per ambiguous charactesr


test_multitrad=defaultdict(list)
test_multisimp=defaultdict(list)
trad_lines=open('../corpora/trad_lines').readlines()
simp_lines=open('../corpora/simp_lines').readlines()
lines_num_trad=[]
lines_num_simp=[]

In [16]:


line_num=0

for line in trad_lines:
        if len(lines_num_trad)==1470:
            break
            
        
        for char_i in range(len(line)):
            char=line[char_i]
            if char not in multitrad:
                continue
            else:
                if len(test_multitrad[char])<20:
                    test_multitrad[char].append({'char_index':char_i,'orig_line_num':line_num,'line':line,'gold':simp_lines[line_num]})
                    lines_num_trad.append(line_num)
        line_num+=1
        

    
line_num=0
for line in simp_lines:
        if len(lines_num_simp)==120:
            break
        for char_i in range(len(line)):
            char=line[char_i]
            if char not in multisimp:
                continue
            else:
                if len(test_multisimp[char])<20:
                    test_multisimp[char].append({'char_index':char_i,'orig_line_num':line_num,'line':line,'gold':trad_lines[line_num]})
                    lines_num_simp.append(line_num)
                    #print (char, line_num,line)
        line_num+=1



defaultdict(list,
            {'並': [{'char_index': 27,
               'gold': '大多数政界人士预测,他将更换声望不佳的总理柯瑞松夫人,并在政府中给予法国声势日增的生态运动党更有份量的地位。\n',
               'line': '大多數政界人士預測,他將更換聲望不佳的總理柯瑞松夫人,並在政府中給予法國聲勢日增的生態運動黨更有份量的地位。\n',
               'orig_line_num': 30},
              {'char_index': 17,
               'gold': '这种趋势还没有完全呈现–法国的选举并未影响国会的组成–而且不会像苏联帝国的崩溃,或是南斯拉夫的分裂那样激烈。\n',
               'line': '這種趨勢還沒有完全呈現–法國的選舉並未影響國會的組成–而且不會像蘇聯帝國的崩潰,或是南斯拉夫的分裂那樣激烈。\n',
               'orig_line_num': 80},
              {'char_index': 46,
               'gold': '「中东新闻报导」指出,利比亚领袖格达费今天表示愿前往华盛顿当面交给布希总统一份能够证明利比亚并未涉及洛克比泛美客机爆炸案的报告。\n',
               'line': '「中東新聞報導」指出,利比亞領袖格達費今天表示願前往華盛頓當面交給布希總統一份能夠證明利比亞並未涉及洛克比泛美客機爆炸案的報告。\n',
               'orig_line_num': 126},
              {'char_index': 46,
               'gold': '联合国安理会今天投票通过对利比亚施以一项空中运输与武器的禁运,阿拉伯联盟愤怒的反应这项表决,并表示,联合国应严惩以色列。\n',
               'line': '聯合國安理會今天投票通過對利比亞施以一項空中運輸與武器的禁運,阿拉伯聯盟憤怒的反應這項表決,並表示,聯合國應嚴懲以色列。\n',
               'orig_line_num': 186},

In [15]:
#store test cases
with open('trad_test.json','w') as f:
    json.dump(test_multitrad,f)
    
with open('simp_test.json', 'w') as f:
    json.dump(test_multisimp, f)


In [13]:
#delete the test sentences from the corpus to form training dataset
line_nums=sorted(set(lines_num_trad+lines_num_simp),reverse=True)

with open('./data/trad_train', 'w') as f:
    for line_num in line_nums:
        del trad_lines[line_num]
    f.writelines(trad_lines)

with open('./data/simp_train', 'w') as f:
    for line_num in line_nums:
        del simp_lines[line_num]
    f.writelines(simp_lines)